In [2]:
# 02_optimizacion_modelos_sms_spam.ipynb

import pandas as pd
import numpy as np

# Scikit-learn y métricas
from sklearn.model_selection       import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline              import Pipeline
from sklearn.preprocessing         import FunctionTransformer, StandardScaler
from sklearn.metrics               import classification_report, confusion_matrix, roc_curve, auc, f1_score

# Clasificadores
from sklearn.linear_model          import LogisticRegression
from sklearn.svm                   import SVC
from sklearn.ensemble              import RandomForestClassifier, VotingClassifier
from sklearn.neighbors             import KNeighborsClassifier
from sklearn.naive_bayes           import MultinomialNB
from sklearn.neural_network        import MLPClassifier
from sklearn.ensemble              import StackingClassifier

# Boosting libraries
import xgboost as xgb
import lightgbm as lgb

import matplotlib.pyplot as plt

# Cargar datos (igual que en el notebook anterior)
ruta = "../datos_raw/SMSSpamCollection"
df = pd.read_csv(ruta, sep="\t", header=None, names=["label", "text"], encoding="utf-8")
df["label_bin"] = df["label"].map({"ham": 0, "spam": 1})

# División inicial en train/test (mantener proporción)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    df["text"], df["label_bin"],
    test_size=0.2,
    random_state=42,
    stratify=df["label_bin"]
)

print("Datos cargados. Tamaño train:", X_train_raw.shape, "tamaño test:", X_test_raw.shape)

Datos cargados. Tamaño train: (4457,) tamaño test: (1115,)
